<a href="https://colab.research.google.com/github/govind527/Pytorch_Practice/blob/main/pytorch_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## importing libraries

import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms

from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

print(torchvision.__version__)

In [ ]:
train_data=datasets.FashionMNIST(
    root="data",# where to download the data
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(), ## how we want to transform data
    target_transform =None ## wnat to transform labels
)

test_data=datasets.FashionMNIST(
    root="data", # where to download the data
    train=False,
    download=True,
    transform=ToTensor(), ## how we want to transform data
    target_transform =None ## wnat to transform labels
)

In [ ]:
len(train_data),len(test_data)

In [ ]:
## first training example
image,label=train_data[0]
image,label

In [ ]:
image.shape,label

In [ ]:
class_names=train_data.classes
class_names

In [ ]:
class_to_id=train_data.class_to_idx
class_to_id

In [ ]:
train_data.targets

## visualizing data

In [ ]:
import matplotlib.pyplot as plt
image,label=train_data[0]
print(image.shape)
plt.imshow(image.squeeze())
plt.title(label);

In [ ]:
plt.imshow(image.squeeze(),cmap="gray")
plt.title(train_data.classes[label])
plt.axis(False);

In [ ]:
##torch.manual_seed(42)
fig=plt.figure(figsize=(9,9))
rows,cols=4,4
for i in range(1,rows*cols+1):
  random_idx=torch.randint(0,len(train_data),size=[1]).item()
  img,label=train_data[random_idx]
  fig.add_subplot(rows,cols,i)
  plt.imshow(img.squeeze(),cmap="gray")
  plt.title(train_data.classes[label])
  plt.axis(False);

## dataloader preparation

In [ ]:
train_data,test_data

In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE=32
train_dataloader=DataLoader(dataset=train_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True)

test_dataloader=DataLoader(dataset=test_data,
                            batch_size=BATCH_SIZE,
                            shuffle=False)

train_dataloader,test_dataloader

In [ ]:
print(f"train Dataloader : {train_dataloader}")
print(f"train Dataloader length : {len(train_dataloader)} of batches {BATCH_SIZE}")
print(f"test Dataloader length : {len(test_dataloader)} of batches {BATCH_SIZE}")

In [ ]:
train_features_batch,train_label_batch=next(iter(train_dataloader))

train_features_batch.shape

In [ ]:
torch.manual_seed(42)
random_idx=torch.randint(0,len(train_features_batch),size=[1]).item()
img,label=train_features_batch[random_idx],train_label_batch[random_idx]
plt.imshow(img.squeeze(),cmap='gray')
plt.title(class_names[label])
plt.axis(False)
print(f"image size: {img.shape}")
print(f"label : {label} || label size: {label.shape}")

## building first baseline model for cv

In [ ]:
from torch import nn
class FashionMNISTModelV0(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               output_shape:int
               ):
    super().__init__()
    self.layer_stack=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape,
                  out_features=hidden_units),
        nn.Linear(in_features=hidden_units,
                  out_features=output_shape),

    )

  def forward(self,x):
    return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)
model_0=FashionMNISTModelV0(
    input_shape=784,
    hidden_units=10,
    output_shape=len(class_names)
).to("cpu")
model_0

In [ ]:
dummy_x=torch.rand([1,1,28,28])
model_0(dummy_x)

In [ ]:
model_0.state_dict()

In [ ]:
import requests
from pathlib import Path

if Path("helper_functions.py").is_file():
  print("helper function already exist skip downloading...")
else:
  print("downloading helper function")
  request=requests.get('https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/helper_functions.py')
  with open('helper_functions.py','wb') as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

## setup loss function and optimizer
loss_fn=nn.CrossEntropyLoss()

optimizer=torch.optim.SGD(params=model_0.parameters(),
                          lr=0.1)

In [ ]:
## creating a funtion to get time of our experiments

from timeit import default_timer as timer

def print_train_time(start:float,
                     end:float,
                     device:torch.device=None):
  total_time=end-start
  print(f"train timr on {device} : {total_time:.3f} seconds")
  return total_time

In [ ]:
from tqdm.auto import tqdm ## to see progress bar

torch.manual_seed(42)
train_time_start_on_cpu=timer()

epochs=3 ## small for faster training time

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch} \n----")
  ## training

  train_loss=0
  for batch, (X,y) in enumerate(train_dataloader):
    model_0.train()

    ## forward pass
    y_pred=model_0(X)

    ## calculate loss

    loss=loss_fn(y_pred,y)
    train_loss+=loss
    ##3. optimizer zero grad

    optimizer.zero_grad()

    ## backward
    loss.backward()

    ##optimizer step

    optimizer.step()

    if batch% 400==0:
      print(f"looked at {batch*len(X)}/{len(train_dataloader.dataset)} samples.")
  train_loss/=len(train_dataloader)


  ##testing

  test_loss,test_acc=0,0
  model_0.eval()
  with torch.inference_mode():
    for X_test, y_test in test_dataloader:
      test_pred=model_0(X_test)

      test_loss+=loss_fn(test_pred,y_test)

      test_acc+=accuracy_fn(y_true=y_test,y_pred=test_pred.argmax(dim=1))

    test_loss/=len(test_dataloader)

    test_acc/=len(test_dataloader)
  print(f"\n Train loss: {train_loss:.4f} | test loss : {test_loss:.4f}, test accuracy : {test_acc:.4f}")

train_time_end_on_cpu=timer()
tottal_train_time_model_0=print_train_time(start=train_time_start_on_cpu,
                                           end=train_time_end_on_cpu,
                                           device=str(next(model_0.parameters()).device))

In [ ]:
str(next(model_0.parameters()).device)

## making predictions and model results

In [ ]:
torch.manual_seed(42)
def eval_model(model:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               accuracy_fn):
  loss,acc=0,0
  model_0.eval()
  with torch.inference_mode():
    for X,y in data_loader:
      ##X,y=X.to(device),y.to(device)
      y_pred=model(X)

      loss+=loss_fn(y_pred,y)
      acc+=accuracy_fn(y_true=y,
                       y_pred=y_pred.argmax(dim=1))

    loss/=len(data_loader)
    acc/=len(data_loader)

  return {"model name" : model.__class__.__name__,
          "model loss" : loss.item(),
          "model acc" : acc}

model_0_results=eval_model(model=model_0,
                           data_loader=test_dataloader,
                           loss_fn=loss_fn,
                           accuracy_fn=accuracy_fn)

model_0_results



In [ ]:
import torch
device ='cuda' if torch.cuda.is_available() else "cpu"
device

In [ ]:
class FashionMNISTModelV1(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               output_shape:int):
    super().__init__()
    self.stack_layer=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape,
                  out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,
                  out_features=output_shape),
        nn.ReLU()
    )
  def forward(self,x:torch.Tensor):
    return self.stack_layer(x)

In [ ]:
torch.manual_seed(42)
model_1=FashionMNISTModelV1(input_shape=784,
                            hidden_units=10,
                            output_shape=len(class_names)).to(device)
model_1

In [ ]:
next(model_1.parameters()).device

In [ ]:
from helper_functions import accuracy_fn

loss_fn=nn.CrossEntropyLoss()

optimizer=torch.optim.SGD(params=model_1.parameters(),
                          lr=0.1)

## writing functionthe for train and test code

In [ ]:

def train_step(model:nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device=device):



  ## training
  model.train()
  train_loss, train_acc=0,0
  for batch, (X,y) in enumerate(data_loader):
    X=X.to(device)
    y=y.to(device)

    ## forward pass
    y_pred=model(X)

    ## calculate loss

    loss=loss_fn(y_pred,y)
    train_loss+=loss

    train_acc+=accuracy_fn(y_true=y,y_pred=y_pred.argmax(dim=1))
    ##3. optimizer zero grad

    optimizer.zero_grad()

    ## backward
    loss.backward()

    ##optimizer step

    optimizer.step()


  train_loss/=len(data_loader)
  train_acc/=len(data_loader)
  print(f"train loss : {train_loss:.5f} | train acc : {train_acc:.2f}%")


In [ ]:

def test_step(model:nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               #optimizer:torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device=device):



  ## test
  model.eval()
  test_loss, test_acc=0,0
  with torch.inference_mode():
    for X,y in data_loader:
      X,y=X.to(device),y.to(device)

      test_pred=model(X)
      test_loss+=loss_fn(test_pred,y)
      test_acc+=accuracy_fn(y_true=y,
                            y_pred=test_pred.argmax(dim=1))

    test_loss/=len(data_loader)
    test_acc/=len(data_loader)

    print(f"test loss : {test_loss:.5f} | test acc : {test_acc:.2f}%")

In [ ]:
torch.manual_seed(42)

from timeit import default_timer as timer

train_time_start_on_gpu=timer()
epochs=3

for epoch in tqdm(range(epochs)):
  print(f"epoch {epoch} \n ====")
  train_step(model=model_1,
             data_loader=train_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device=device)

  train_step(model=model_1,
             data_loader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device=device)

train_time_end_on_gpu=timer()

total_train_time_model_1=print_train_time(start=train_time_start_on_gpu,
                                          end=train_time_end_on_gpu,
                                          device=device)


In [ ]:
torch.manual_seed(42)
def eval_model(model:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               accuracy_fn,
               device=device):
  loss,acc=0,0
  model_0.eval()
  with torch.inference_mode():
    for X,y in data_loader:
      X,y=X.to(device),y.to(device)
      y_pred=model(X)

      loss+=loss_fn(y_pred,y)
      acc+=accuracy_fn(y_true=y,
                       y_pred=y_pred.argmax(dim=1))

    loss/=len(data_loader)
    acc/=len(data_loader)

  return {"model name" : model.__class__.__name__,
          "model loss" : loss.item(),
          "model acc" : acc}


In [ ]:
model_1_results=eval_model(model=model_1,
                           data_loader=test_dataloader,
                           loss_fn=loss_fn,
                           accuracy_fn=accuracy_fn,
                           device=device)
model_1_results

In [ ]:
model_0_results

# Building a Convolutional Neural Network (CNN)

In [ ]:
class FashionMNISTModelV2(nn.Module):
  """
  Model architecture that replicates the TinyVGG
  """
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               output_shape:int):
    super().__init__()
    self.conv_block_1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),

        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_2=nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*7*7,## trick to calculate
                  out_features=output_shape)
    )

  def forward(self,x):
    x=self.conv_block_1(x)
    #print(x.shape)
    x=self.conv_block_2(x)
    #print(x.shape)
    x=self.classifier(x)
    return x



In [ ]:
torch.manual_seed(42)
model_2=FashionMNISTModelV2(input_shape=1,
                            hidden_units=10,
                            output_shape=len(class_names)).to(device)

model_2

In [ ]:
torch.manual_seed(42)

images=torch.randn(size=(32,3,64,64))

test_image=images[0]
images.shape,test_image.shape

In [ ]:
conv_layer=nn.Conv2d(in_channels=3,
                     out_channels=10,
                     kernel_size=3,
                     stride=1,
                     padding=0)
conv_output=conv_layer(test_image)
conv_output.shape

In [ ]:
rand_image_tensor=torch.randn(size=(1,28,28))
model_2(rand_image_tensor.unsqueeze(0).to(device))

In [ ]:
model_2(image.unsqueeze(0).to(device))

In [ ]:
from helper_functions import accuracy_fn
loss_fn=nn.CrossEntropyLoss()

optimizer=torch.optim.SGD(params=model_2.parameters(),lr=0.1)

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from timeit import default_timer as timer

train_time_start_model_2=timer()

epochs=3
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch} \n ---->")
  train_step(model=model_2,
             data_loader=train_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device=device)
  test_step(model=model_2,
             data_loader=test_dataloader,
             loss_fn=loss_fn,
             accuracy_fn=accuracy_fn,
             device=device)

train_time_end_model_2=timer()
total_train_time_model_2=print_train_time(start=train_time_start_model_2,
                                          end=train_time_end_model_2,
                                          device=device)

In [ ]:
model_2_results=eval_model(model=model_2,
                           data_loader=test_dataloader,
                           loss_fn=loss_fn,
                           accuracy_fn=accuracy_fn,
                           device=device)

model_2_results

In [ ]:
import pandas as pd
compare_results=pd.DataFrame([model_0_results,
                              model_1_results,
                              model_2_results
                              ])

compare_results

In [ ]:
compare_results.set_index("model name")["model acc"].plot(kind='barh')
plt.xlabel('accuracy (%)')
plt.ylabel("model")

In [ ]:
!nvidia-smi

In [ ]:
def make_predition(model:torch.nn.Module,
                   data:list,
                   device:torch.device=device):
  pred_probs=[]
  model.to(device)
  model.eval()
  with torch.inference_mode():
    for sample in data:
      sample=torch.unsqueeze(sample,dim=0).to(device)
      pred_logit=model(sample)
      pred_prob=torch.softmax(pred_logit.squeeze(),dim=0)

      pred_probs.append(pred_prob.cpu())
  return torch.stack(pred_probs)


import random
random.seed(42)
test_samples=[]
test_labels=[]
for sample,label in random.sample(list(test_data),k=9):
  test_samples.append(sample)
  test_labels.append(label)
test_samples[0].shape

In [ ]:
plt.imshow(test_samples[0].squeeze(),cmap='gray')
plt.title(class_names[test_labels[0]])

In [ ]:
pred_probs=make_predition(model=model_2,data=test_samples)

pred_probs[:2]

In [ ]:
pred_classes=pred_probs.argmax(dim=1)
pred_classes

In [ ]:
plt.figure(figsize=(9,9))
nrows=3
ncols=3
for i,sample in enumerate(test_samples):
  plt.subplot(nrows,ncols,i+1)
  plt.imshow(sample.squeeze(),cmap='gray')

  pred_label=class_names[pred_classes[i]]

  truth_label=class_names[test_labels[i]]

  title_text =f"Pred: {pred_label} | truth : {truth_label}"

  if pred_label==truth_label:
    plt.title(title_text,fontsize=10,c='g')
  else:
    plt.title(title_text,fontsize=10,c='r')

  plt.axis(False);

## making confusion matrix

In [ ]:
#import torchmetrics
from tqdm.auto import tqdm
import mlxtend


y_preds=[]
model_2.eval()
with torch.inference_mode():
  for X,y in tqdm(test_dataloader,desc="Making Predictions..."):
    X,y=X.to(device),y.to(device)

    y_logit=model_2(X)
    y_pred=torch.softmax(y_logit.squeeze(),dim=0).argmax(dim=1)

    y_preds.append(y_pred.cpu())

##print(y_preds)
y_pred_tensor=torch.cat(y_preds)
y_pred_tensor

In [ ]:
len(y_pred_tensor)

In [ ]:
try:
  import torchmetrics,mlxtend
  print(f"mlxtend version : {mlxtend.__version__}")
  assert int(mlxtend.__version__.split(".")[1]>=19, "mlxtend version should be greater than 19")

except:
  !pip install -q torchmetrics -U mlxtend
  import torchmetrics,mlxtend
  print(f"mlxtend version : {mlxtend.__version__}")

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

confmat=ConfusionMatrix(task='multiclass', num_classes=len(class_names))
confmat_tensor=confmat(preds=y_pred_tensor,
                       target=test_data.targets)

fig,ax=plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(),
    class_names=class_names,
    figsize=(7,7)

)

In [ ]:
confmat_tensor

#Saving model and loading model

In [ ]:
from pathlib import Path

MODEL_PATH=Path('Models')
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)

MODEL_NAME="03_pytorch_computer_vision_model_2.pth"
MODEL_SAVE_PATH=MODEL_PATH/MODEL_NAME

print(f"saving model : {MODEL_SAVE_PATH}")
torch.save(obj=model_2.state_dict(),
           f=MODEL_SAVE_PATH)

In [ ]:
torch.manual_seed(42)
laoded_model_2=FashionMNISTModelV2(input_shape=1,
                                   hidden_units=10,
                                   output_shape=len(class_names))

laoded_model_2.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

laoded_model_2.to(device)

In [ ]:
model_2_results

In [ ]:
torch.manual_seed(42)
loaded_model_2_results=eval_model(
    model=laoded_model_2,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn
)

loaded_model_2_results